# CHAPTER 9 - Quantum Support Vector Machines - Code

*Note*: You may skip the following four cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [1]:
#!pip install scikit-learn
#!pip install pennylane==0.26
#!pip install qiskit==0.39.2
#!pip install qiskit_machine_learning==0.5.0

In [8]:
import numpy as np

print(np.__version__)
seed = 1234
np.random.seed(seed)

2.0.1


In [9]:
from sklearn.datasets import load_wine

x,y = load_wine(return_X_y = True)

In [10]:
x = x[:59+71]
y = y[:59+71]

In [11]:
from sklearn.model_selection import train_test_split

x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9)

In [12]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

In [13]:
x_test = scaler.transform(x_test)
x_test = np.clip(x_test,0,1)

In [14]:
import pennylane as qml
print(qml.__version__)

nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires = range(nqubits))

0.42.0


In [15]:
from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel = qkernel).fit(x_tr, y_tr)

In [16]:
from sklearn.metrics import accuracy_score

print(accuracy_score(svm.predict(x_test), y_test))

0.9230769230769231


In [17]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 8)

xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

In [18]:
nqubits = 8
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AngleEmbedding(a, wires=range(nqubits))
    qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
    return qml.probs(wires = range(nqubits))

In [19]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)

print(accuracy_score(svm.predict(xs_test), y_test))

1.0


In [20]:
from itertools import combinations

def ZZFeatureMap(nqubits, data):

    # Number of variables that we will load:
    # could be smaller than the number of qubits.
    nload = min(len(data), nqubits) 

    for i in range(nload):
        qml.Hadamard(i)
        qml.RZ(2.0 * data[i], wires = i)

    for pair in list(combinations(range(nload), 2)):
        q0 = pair[0]
        q1 = pair[1]

        qml.CZ(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - data[q0]) *
            (np.pi - data[q1]), wires = q1)
        qml.CZ(wires = [q0, q1])

In [21]:
nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    ZZFeatureMap(nqubits, a)
    qml.adjoint(ZZFeatureMap)(nqubits, b)
    return qml.probs(wires = range(nqubits))

svm = SVC(kernel=qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))

0.8461538461538461


In [22]:
from sklearn.metrics import accuracy_score

In [23]:
from qiskit import *
import qiskit
print(qiskit.__version__)

1.2.4


In [24]:
from qiskit.circuit import Parameter
parameter = Parameter("x")
qc = QuantumCircuit(1)
qc.rx(parameter, 0)

In [18]:
from qiskit.circuit import ParameterVector
parameter = ParameterVector("x", length = 2)
qc = QuantumCircuit(2)
qc.rx(parameter[0], 0)
qc.rx(parameter[1], 1)

In [19]:
from qiskit.circuit.library import ZZFeatureMap #Update
#from qiskit.circuit.library import zz_feature_map #Update

zzfm = ZZFeatureMap(8)

In [20]:
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler as Sampler
import qiskit_machine_learning
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
print(qiskit_machine_learning.__version__)

0.8.3


In [21]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_aer import AerSimulator

sampler = Sampler()
fidelity_object = ComputeUncompute(sampler=sampler)

qkernel = FidelityQuantumKernel(feature_map = zzfm, fidelity = fidelity_object) #update

In [22]:
from qiskit_machine_learning.algorithms import QSVC

qsvm = QSVC(quantum_kernel = qkernel, C=10)
qsvm.fit(xs_tr, y_tr)

QSVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.fidelity_quantum_kernel.FidelityQuantumKernel object at 0x000001600AADB5C0>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [23]:
print(accuracy_score(qsvm.predict(xs_test), y_test))

0.6923076923076923


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
provider = IBMQ.load_account()

In [ ]:
from qiskit.providers.ibmq import *

dev_list = provider.backends(
    filters = lambda x: x.configuration().n_qubits >= 7,
    simulator = False)
dev = least_busy(dev_list)

In [27]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 7)

xss_tr = pca.fit_transform(x_tr)

xss_test = pca.transform(x_test)

In [ ]:
zzfm = ZZFeatureMap(7)
qkernel = QuantumKernel(feature_map = zzfm, quantum_instance = dev)
qsvm = QSVC(quantum_kernel = qkernel)
qsvm.fit(xss_tr, y_tr)

In [38]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

In [40]:
from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token='VMgmA9Gjyklr5uAIq8S6FYXrmKvoVN3QLo714I6KjRBA',
    channel="ibm_cloud",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/d860b40ee4e144ca8d380d1a5d0f44a1:26fb1ed2-ea31-4963-a029-04765df71247::",  # Optional
    name="Lei Qian",
    overwrite=True
)

service = QiskitRuntimeService()

In [41]:
backend_channel = "ibm_brisbane"
backend = service.backend(name = backend_channel)

In [42]:
from qiskit_ibm_runtime import Session, Sampler

with Session(service=service, backend=backend) as session:
    sampler = Sampler(session=session, options={'backend_name': backend.name()})


C:\Users\leqian\AppData\Local\Temp\ipykernel_62660\2976092248.py:3: DeprecationWarning: The service parameter is deprecated as of qiskit-ibm-runtime 0.26.0 and will be removed no sooner than 3 months after the release date. The service can be extracted from the backend object so it is no longer necessary.
  with Session(service=service, backend=backend) as session:


RequestsApiError: '400 Client Error: Bad Request for url: https://us-east.quantum-computing.cloud.ibm.com/sessions. {"errors":[{"code":1352,"message":"You are not authorized to run a session when using the open plan.","solution":"Create an instance of a different plan type or use a different execution mode <https://quantum.cloud.ibm.com/docs/guides/execution-modes>.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"c0dcbb8f-092b-48cb-9b35-fe4b55b16870"}\n'

In [34]:

# Create the feature map circuit
num_features = 7  # We're using 4 features
zzfm = ZZFeatureMap(num_features)

# Create the quantum kernel with the real quantum backend
quantum_kernel = FidelityQuantumKernel(
    feature_map=zzfm,
    quantum_instance=backend
)



TypeError: FidelityQuantumKernel.__init__() got an unexpected keyword argument 'quantum_instance'

In [ ]:
# Create and train the QSVC
qsvc = QSVC(
    quantum_kernel=quantum_kernel,
    C=1.0  # Regularization parameter
)

# Train the model
print("Training the quantum SVM...")
qsvc.fit(X_train_normalized, y_train)

# Make predictions
y_pred = qsvc.predict(X_test_normalized)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"Test accuracy: {accuracy:.4f}")

# Print some additional metrics
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))